In [2]:
# Qiwi Card
# 1. Importing necessary libraries
# 2. Reading csv file via Pandas
# 3. Before presenting the data, we are telling it which columns it should read in order to save time and use only necessary columns
# 4. We can also change the column type ( That will help us for doing the look up)
# 5. We are reading csv file because excel files are usually bigger in size and take longer to read
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
parse_dates = ["UPDATE TIME","ORDER TIME"]
internal = pd.read_excel("C:\\Users\\XCG\\Desktop\\Recons_all_for_Drive\\Qiwi_RUB_Card\\6 months 2023\\Card\\Card_Acquiring_data_2023_7 months_Internal.xlsx",
usecols = ["TRANSACTION ID",'PAY CHANNEL',"METHOD",'UPDATE TIME','ORDER TIME','PAY AMOUNT'],dtype ={"TRANSACTION ID": object}, parse_dates = parse_dates)


In [3]:
# 1. We are checking whether type of columns are relevant (for instance, some of the columns might have numbers but those numbers were saved as text)
# By checking types of columns, we can fix aforementioned mistakes beforehand and save time.
internal.dtypes

TRANSACTION ID            object
METHOD                    object
PAY CHANNEL               object
PAY AMOUNT               float64
ORDER TIME        datetime64[ns]
UPDATE TIME       datetime64[ns]
dtype: object

In [5]:
internal.tail()
# I am reading the first five rows of my data 

,TRANSACTION ID,METHOD,PAY CHANNEL,PAY AMOUNT,ORDER TIME,UPDATE TIME
679702,2023010100022900939,CreditCard,CreditCard - QiWi - RUB(redirect),462.50,2023-01-01 00:02:29,2023-01-01 00:03:42
679703,2023010100021700882,CreditCard,CreditCard - QiWi - RUB(redirect),138.75,2023-01-01 00:02:17,2023-01-01 00:02:57
679704,2023010100015700764,CreditCard,CreditCard - QiWi - RUB(redirect),92.50,2023-01-01 00:01:57,2023-01-01 00:04:23
679705,2022123123580207514,CreditCard,CreditCard - QiWi - RUB(redirect),9.25,2022-12-31 23:58:02,2023-01-01 00:00:51
679706,2022123123550106478,CreditCard,CreditCard - QiWi - RUB(redirect),277.50,2022-12-31 23:55:01,2023-01-01 00:01:41


In [6]:
# Let's check whether we have any missing data in our file 
internal.isna().any().any()

False

In [7]:
# Segregate hours from dates in a new column
internal["DateOnly"] = internal["UPDATE TIME"].dt.date
internal["DateOnly"].value_counts()

2023-04-20    16314
2023-04-08    15412
2023-02-04    14339
2023-03-25    14220
2023-02-25    13718
              ...  
2023-06-26     1289
2023-06-25     1266
2023-06-29     1261
2023-04-27        1
2023-04-28        1
Name: DateOnly, Length: 207, dtype: int64

In [11]:
# 2. Reading excel file via Pandas (because it was downloaded as an excel file, will download and read csv to save time)
# 3. Before presenting the data, we are telling it which columns it should read in order to save time and use only necessary columns
# 4. We can also change the column type that we will need soon for matching ( That will help us for doing the look up)
parse_dates = ["Created (UTC+3)","Processed (UTC+3)"]
external = pd.read_excel("C:\\Users\\XCG\\Desktop\\August_23\\Recons_\\Qiwi Card\\6 months 2023\\External Data 01-06.23 CC Qiwi\\January_External_23_CC.xlsx",
usecols = ["Shop id", "Created (UTC+3)","Processed (UTC+3)","Type","Shop transaction id","Payway","Shop fee","Shop refund","Payer price", "Status"],
dtype ={"Shop transaction id": object},parse_dates = parse_dates)

In [12]:
external.head()

,Shop id,Created (UTC+3),Processed (UTC+3),Type,Shop transaction id,Payway,Shop fee,Shop refund,Payer price,Status
0,405,2023-01-31 23:59:09,2023-01-31 23:59:18,invoices,2023013120590502292,qiwi_wallet_rub,4.62,87.88,92.50,Successful
1,405,2023-01-31 23:58:32,2023-01-31 23:59:07,invoices,2023013120582601631,qiwi_wallet_rub,5.00,95.00,100.00,Successful
2,406,2023-01-31 23:56:35,2023-01-31 23:58:35,invoices,2023013120562809708,card_acquiring_rub,0.46,8.79,9.25,Rejected
3,406,2023-01-31 23:48:37,2023-01-31 23:58:31,invoices,2023013120465100209,card_acquiring_rub,179.95,3419.05,3599.00,Rejected
4,405,2023-01-31 23:57:28,2023-01-31 23:57:49,invoices,2023013120571600457,qiwi_wallet_rub,2.50,47.50,50.00,Successful


In [13]:
external.dtypes

Shop id                         int64
Created (UTC+3)        datetime64[ns]
Processed (UTC+3)      datetime64[ns]
Type                           object
Shop transaction id            object
Payway                         object
Shop fee                      float64
Shop refund                   float64
Payer price                   float64
Status                         object
dtype: object

In [14]:
# We do have a missing value in our DF
# Let's see which column has a missing value and find out the reason for it 
nan_locations = external[external.isna().any(axis = 1)]
print("Rows with NaN values:")
print(nan_locations)

# As we see, all our missing values are related to chargebacks

Rows with NaN values:
        Shop id     Created (UTC+3)   Processed (UTC+3)        Type  \
124361      406 2023-01-16 16:22:14 2023-01-16 16:22:14  chargeback   
160457      406 2023-01-11 17:49:45 2023-01-11 17:49:45  chargeback   
160461      406 2023-01-11 17:49:04 2023-01-11 17:49:04  chargeback   

       Shop transaction id Payway  Shop fee  Shop refund  Payer price  \
124361                 NaN    NaN       NaN      -3599.0          NaN   
160457                 NaN    NaN       NaN      -1850.0          NaN   
160461                 NaN    NaN       NaN      -1850.0          NaN   

            Status  
124361  Successful  
160457  Successful  
160461  Successful  


In [15]:
# Let's create a new column and have only dates without hours
external["DateOnly"] = external["Processed (UTC+3)"].dt.date
external.head()
#external["DateOnly"].value_counts()

,Shop id,Created (UTC+3),Processed (UTC+3),Type,Shop transaction id,Payway,Shop fee,Shop refund,Payer price,Status,DateOnly
0,405,2023-01-31 23:59:09,2023-01-31 23:59:18,invoices,2023013120590502292,qiwi_wallet_rub,4.62,87.88,92.50,Successful,2023-01-31
1,405,2023-01-31 23:58:32,2023-01-31 23:59:07,invoices,2023013120582601631,qiwi_wallet_rub,5.00,95.00,100.00,Successful,2023-01-31
2,406,2023-01-31 23:56:35,2023-01-31 23:58:35,invoices,2023013120562809708,card_acquiring_rub,0.46,8.79,9.25,Rejected,2023-01-31
3,406,2023-01-31 23:48:37,2023-01-31 23:58:31,invoices,2023013120465100209,card_acquiring_rub,179.95,3419.05,3599.00,Rejected,2023-01-31
4,405,2023-01-31 23:57:28,2023-01-31 23:57:49,invoices,2023013120571600457,qiwi_wallet_rub,2.50,47.50,50.00,Successful,2023-01-31


In [113]:
#import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# Create a heatmap to visualize NaN values
#plt.figure(figsize=(10, 6))
#sns.heatmap(external.isna(), cmap='viridis', cbar=False)
#plt.title('NaN Values in DataFrame')
#plt.show()

In [16]:
# We need to make our column names the same in order to be able to match them
external.rename(columns={"Shop transaction id":"TRANSACTION ID"},inplace = True)
external = external[(external["Status"] == "Successful")]

In [17]:
external.head()
#external["Status"].value_counts()

,Shop id,Created (UTC+3),Processed (UTC+3),Type,TRANSACTION ID,Payway,Shop fee,Shop refund,Payer price,Status,DateOnly
0,405,2023-01-31 23:59:09,2023-01-31 23:59:18,invoices,2023013120590502292,qiwi_wallet_rub,4.62,87.88,92.5,Successful,2023-01-31
1,405,2023-01-31 23:58:32,2023-01-31 23:59:07,invoices,2023013120582601631,qiwi_wallet_rub,5.00,95.00,100.0,Successful,2023-01-31
4,405,2023-01-31 23:57:28,2023-01-31 23:57:49,invoices,2023013120571600457,qiwi_wallet_rub,2.50,47.50,50.0,Successful,2023-01-31
5,405,2023-01-31 23:57:14,2023-01-31 23:57:45,invoices,2023013120571000384,qiwi_wallet_rub,29.40,558.60,588.0,Successful,2023-01-31
6,406,2023-01-31 23:56:01,2023-01-31 23:57:32,invoices,2023013120550608368,card_acquiring_rub,199.95,3799.05,3999.0,Successful,2023-01-31


In [18]:
# Vlook up (Merging)
merged_internal = pd.merge(internal,external, on = "TRANSACTION ID", how = "outer")
merged_internal.head()

,TRANSACTION ID,METHOD,PAY CHANNEL,PAY AMOUNT,ORDER TIME,UPDATE TIME,DateOnly_x,Shop id,Created (UTC+3),Processed (UTC+3),Type,Payway,Shop fee,Shop refund,Payer price,Status,DateOnly_y
0,2023073123575508934,CreditCard,CreditCard - QiWi - RUB(redirect),9.5,2023-07-31 23:57:55,2023-07-31 23:58:56,2023-07-31,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023073123575208852,CreditCard,CreditCard - QiWi - RUB(redirect),142.5,2023-07-31 23:57:52,2023-07-31 23:58:57,2023-07-31,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023073123564907642,CreditCard,CreditCard - QiWi - RUB(redirect),285.0,2023-07-31 23:56:49,2023-07-31 23:57:48,2023-07-31,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023073123552105997,CreditCard,CreditCard - QiWi - RUB(redirect),123.5,2023-07-31 23:55:21,2023-07-31 23:56:16,2023-07-31,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023073123524103078,CreditCard,CreditCard - QiWi - RUB(redirect),950.0,2023-07-31 23:52:41,2023-07-31 23:53:43,2023-07-31,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
# Now, let's filter only those dates that we needed to reconcile
# The reason we are doing that is in order to fix problems with Date difference, we tend to download +1 day and -1 day of the actual
# Reconciliation dates and that helps us to find even those transactions that were updated in a different time
# Thus,in order to move back to the actual dates of reconciliation, we will have to filter those specific dates


In [19]:
filtered_internal = merged_internal[(merged_internal['UPDATE TIME'] >= "2023-01-01") & (merged_internal['UPDATE TIME'] < "2023-01-31")]
# We are making a new data frame by filtering only dates that we need for reconciliation
filtered_internal.head()

,TRANSACTION ID,METHOD,PAY CHANNEL,PAY AMOUNT,ORDER TIME,UPDATE TIME,DateOnly_x,Shop id,Created (UTC+3),Processed (UTC+3),Type,Payway,Shop fee,Shop refund,Payer price,Status,DateOnly_y
575859,2023013023584203181,CreditCard,CreditCard - QiWi - RUB(redirect),9.25,2023-01-30 23:58:42,2023-01-30 23:59:31,2023-01-30,406.0,2023-01-31 02:58:50,2023-01-31 02:59:20,invoices,card_acquiring_rub,0.46,8.79,9.25,Successful,2023-01-31
575860,2023013023530108523,CreditCard,CreditCard - QiWi - RUB(redirect),277.50,2023-01-30 23:53:01,2023-01-30 23:54:42,2023-01-30,406.0,2023-01-31 02:53:21,2023-01-31 02:54:31,invoices,card_acquiring_rub,13.88,263.62,277.50,Successful,2023-01-31
575861,2023013023504106449,CreditCard,CreditCard - QiWi - RUB(redirect),462.50,2023-01-30 23:50:41,2023-01-30 23:51:45,2023-01-30,406.0,2023-01-31 02:51:00,2023-01-31 02:51:34,invoices,card_acquiring_rub,23.12,439.38,462.50,Successful,2023-01-31
575862,2023013023495705847,CreditCard,CreditCard - QiWi - RUB(redirect),55.00,2023-01-30 23:49:57,2023-01-30 23:51:31,2023-01-30,406.0,2023-01-31 02:50:07,2023-01-31 02:51:18,invoices,card_acquiring_rub,2.75,52.25,55.00,Successful,2023-01-31
575863,2023013023475604139,CreditCard,CreditCard - QiWi - RUB(redirect),445.00,2023-01-30 23:47:56,2023-01-30 23:48:59,2023-01-30,406.0,2023-01-31 02:48:09,2023-01-31 02:48:48,invoices,card_acquiring_rub,22.25,422.75,445.00,Successful,2023-01-31


In [23]:
#Read new_data
#filtered_internal.head()
filtered_internal["DateOnly_x"].value_counts()

2023-01-28    13424
2023-01-01     6932
2023-01-21     6469
2023-01-14     5451
2023-01-29     4512
2023-01-02     4464
2023-01-16     3816
2023-01-05     3718
2023-01-30     3527
2023-01-22     3525
2023-01-20     3445
2023-01-07     3099
2023-01-15     3067
2023-01-06     2923
2023-01-17     2766
2023-01-08     2739
2023-01-03     2722
2023-01-18     2623
2023-01-19     2580
2023-01-23     2470
2023-01-13     2461
2023-01-10     2445
2023-01-09     2131
2023-01-11     2102
2023-01-04     2037
2023-01-12     2008
2023-01-25     1776
2023-01-27     1611
2023-01-24     1523
2023-01-26     1482
Name: DateOnly_x, dtype: int64

In [24]:
# Let's check again whether we have any nan_values (missing datas or maybe mismatches)
filtered_internal.isna().any().any()
#We do have it. Let's see it!

True

In [25]:
# Checking the Nan Data
nan_internal = filtered_internal[filtered_internal.isna().any(axis = 1)]
print("Rows with NaN values:")
print(nan_internal)
# As we see, there is only one mismatch and the reason for that is its status is (F)

Rows with NaN values:
             TRANSACTION ID      METHOD                        PAY CHANNEL  \
679512  2022123111563604300  CreditCard  CreditCard - QiWi - RUB(redirect)   
679513  2022123111482908413  CreditCard  CreditCard - QiWi - RUB(redirect)   

        PAY AMOUNT          ORDER TIME         UPDATE TIME  DateOnly_x  \
679512       146.0 2022-12-31 11:56:36 2023-01-01 02:42:31  2023-01-01   
679513       146.0 2022-12-31 11:48:29 2023-01-01 02:39:37  2023-01-01   

        Shop id Created (UTC+3) Processed (UTC+3) Type Payway  Shop fee  \
679512      NaN             NaT               NaT  NaN    NaN       NaN   
679513      NaN             NaT               NaT  NaN    NaN       NaN   

        Shop refund  Payer price Status DateOnly_y  
679512          NaN          NaN    NaN        NaN  
679513          NaN          NaN    NaN        NaN  


In [26]:
# Saving in new variable for knowing the exact reason for our job!!!
merged_internal_final = filtered_internal

In [27]:
merged_internal_final["Diff"] = merged_internal_final["Payer price"] - merged_internal_final["PAY AMOUNT"]
merged_internal_final.head()

C:\Users\XCG\AppData\Local\Temp\ipykernel_29744\1802694572.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_internal_final["Diff"] = merged_internal_final["Payer price"] - merged_internal_final["PAY AMOUNT"]


,TRANSACTION ID,METHOD,PAY CHANNEL,PAY AMOUNT,ORDER TIME,UPDATE TIME,DateOnly_x,Shop id,Created (UTC+3),Processed (UTC+3),Type,Payway,Shop fee,Shop refund,Payer price,Status,DateOnly_y,Diff
575859,2023013023584203181,CreditCard,CreditCard - QiWi - RUB(redirect),9.25,2023-01-30 23:58:42,2023-01-30 23:59:31,2023-01-30,406.0,2023-01-31 02:58:50,2023-01-31 02:59:20,invoices,card_acquiring_rub,0.46,8.79,9.25,Successful,2023-01-31,0.0
575860,2023013023530108523,CreditCard,CreditCard - QiWi - RUB(redirect),277.50,2023-01-30 23:53:01,2023-01-30 23:54:42,2023-01-30,406.0,2023-01-31 02:53:21,2023-01-31 02:54:31,invoices,card_acquiring_rub,13.88,263.62,277.50,Successful,2023-01-31,0.0
575861,2023013023504106449,CreditCard,CreditCard - QiWi - RUB(redirect),462.50,2023-01-30 23:50:41,2023-01-30 23:51:45,2023-01-30,406.0,2023-01-31 02:51:00,2023-01-31 02:51:34,invoices,card_acquiring_rub,23.12,439.38,462.50,Successful,2023-01-31,0.0
575862,2023013023495705847,CreditCard,CreditCard - QiWi - RUB(redirect),55.00,2023-01-30 23:49:57,2023-01-30 23:51:31,2023-01-30,406.0,2023-01-31 02:50:07,2023-01-31 02:51:18,invoices,card_acquiring_rub,2.75,52.25,55.00,Successful,2023-01-31,0.0
575863,2023013023475604139,CreditCard,CreditCard - QiWi - RUB(redirect),445.00,2023-01-30 23:47:56,2023-01-30 23:48:59,2023-01-30,406.0,2023-01-31 02:48:09,2023-01-31 02:48:48,invoices,card_acquiring_rub,22.25,422.75,445.00,Successful,2023-01-31,0.0


In [28]:
# No Difference between Payer Price and Payamount
merged_internal_final["Diff"].sum()

0.0

In [29]:
#Now let's move this file into excel file so it can be readable by users
# We can also have it csv
merged_internal_final.to_excel("C:\\Users\\XCG\\Desktop\\For Automation\\Work_Automation\\_int_qiwi_cardjan_automated_recon.xlsx", index = False)
#merged_internal_final.to_csv("C:\\Users\\XCG\\Desktop\\For Automation\\Work_Automation\\_int_qiwi_automated_2_csv.csv", index = False)

In [30]:
# Now let's make merging from external to internal
merged_external = pd.merge(external,internal,on = "TRANSACTION ID", how = "outer")
merged_external.head()

,Shop id,Created (UTC+3),Processed (UTC+3),Type,TRANSACTION ID,Payway,Shop fee,Shop refund,Payer price,Status,DateOnly_x,METHOD,PAY CHANNEL,PAY AMOUNT,ORDER TIME,UPDATE TIME,DateOnly_y
0,405.0,2023-01-31 23:59:09,2023-01-31 23:59:18,invoices,2023013120590502292,qiwi_wallet_rub,4.62,87.88,92.5,Successful,2023-01-31,NaN,NaN,NaN,NaT,NaT,NaN
1,405.0,2023-01-31 23:58:32,2023-01-31 23:59:07,invoices,2023013120582601631,qiwi_wallet_rub,5.00,95.00,100.0,Successful,2023-01-31,NaN,NaN,NaN,NaT,NaT,NaN
2,405.0,2023-01-31 23:57:28,2023-01-31 23:57:49,invoices,2023013120571600457,qiwi_wallet_rub,2.50,47.50,50.0,Successful,2023-01-31,NaN,NaN,NaN,NaT,NaT,NaN
3,405.0,2023-01-31 23:57:14,2023-01-31 23:57:45,invoices,2023013120571000384,qiwi_wallet_rub,29.40,558.60,588.0,Successful,2023-01-31,NaN,NaN,NaN,NaT,NaT,NaN
4,406.0,2023-01-31 23:56:01,2023-01-31 23:57:32,invoices,2023013120550608368,card_acquiring_rub,199.95,3799.05,3999.0,Successful,2023-01-31,CreditCard,CreditCard - QiWi - RUB(redirect),3999.0,2023-01-31 20:55:06,2023-01-31 20:57:44,2023-01-31


In [33]:
# Let's filter actual dates for our needed reconciliation dates
filtered_external = merged_external[(merged_external["Processed (UTC+3)"] >= "2023-01-01") & (merged_external["Processed (UTC+3)"] <= "2023-01-31")]

In [37]:
filtered_external["DateOnly_x"].value_counts()
#filtered_external.head()

2023-01-28    24363
2023-01-01    13615
2023-01-21    12128
2023-01-14    10380
2023-01-02     9348
2023-01-29     9008
2023-01-16     7505
2023-01-05     7009
2023-01-30     6943
2023-01-22     6794
2023-01-20     6635
2023-01-06     6413
2023-01-15     6298
2023-01-07     6291
2023-01-17     5823
2023-01-03     5664
2023-01-08     5495
2023-01-18     5479
2023-01-19     5219
2023-01-13     5202
2023-01-23     5040
2023-01-10     4905
2023-01-11     4708
2023-01-09     4613
2023-01-04     4612
2023-01-12     4139
2023-01-25     3602
2023-01-27     3558
2023-01-24     2992
2023-01-26     2880
Name: DateOnly_x, dtype: int64

In [38]:
# Let's check for Nan Values on our new Merged file
# Checking the Nan Data
nan_external = filtered_external[filtered_external.isna().any(axis = 1)]
print("Rows with NaN values:")
print(nan_external)
# As we see, there is only one mismatch and the reason for that is it is a chargeback

Rows with NaN values:
        Shop id     Created (UTC+3)   Processed (UTC+3)      Type  \
3997      405.0 2023-01-30 23:55:54 2023-01-30 23:59:14  invoices   
3998      405.0 2023-01-30 23:58:58 2023-01-30 23:59:09  invoices   
4005      405.0 2023-01-30 23:55:49 2023-01-30 23:56:24  invoices   
4007      405.0 2023-01-30 23:55:41 2023-01-30 23:56:06  invoices   
4012      405.0 2023-01-30 23:54:45 2023-01-30 23:55:16  invoices   
...         ...                 ...                 ...       ...   
210652    406.0 2023-01-01 00:01:09 2023-01-01 00:01:35  invoices   
210653    405.0 2023-01-01 00:01:23 2023-01-01 00:01:31  invoices   
210654    405.0 2023-01-01 00:00:53 2023-01-01 00:01:04  invoices   
210655    406.0 2023-01-01 00:00:01 2023-01-01 00:01:00  invoices   
210656    405.0 2023-01-01 00:00:13 2023-01-01 00:00:51  invoices   

             TRANSACTION ID              Payway  Shop fee  Shop refund  \
3997    2023013020555005765     qiwi_wallet_rub     40.00       760.00   
3

In [130]:
nan_external.isna().sum()

Shop id              0
Created (UTC+3)      0
Processed (UTC+3)    0
Type                 0
TRANSACTION ID       0
Payway               0
Shop fee             2
Shop refund          0
Payer price          0
Status               0
DateOnly_x           0
METHOD               5
PAY CHANNEL          5
PAY AMOUNT           5
ORDER TIME           5
UPDATE TIME          5
DateOnly_y           5
dtype: int64

In [131]:
# Saving our file in the new file
merged_external_final = filtered_external
merged_external_final.head()

,Shop id,Created (UTC+3),Processed (UTC+3),Type,TRANSACTION ID,Payway,Shop fee,Shop refund,Payer price,Status,DateOnly_x,METHOD,PAY CHANNEL,PAY AMOUNT,ORDER TIME,UPDATE TIME,DateOnly_y
877,406.0,2023-11-20 23:58:53,2023-11-20 23:59:16,invoices,2023112020583400441,card_acquiring_rub,21.71,460.79,482.5,Successful,2023-11-20,CreditCard,CreditCard - QiWi - RUB(redirect),482.5,2023-11-20 20:58:34,2023-11-20 20:59:27,2023-11-20
878,406.0,2023-11-20 23:58:38,2023-11-20 23:59:11,invoices,2023112020582900372,card_acquiring_rub,13.03,276.47,289.5,Successful,2023-11-20,CreditCard,CreditCard - QiWi - RUB(redirect),289.5,2023-11-20 20:58:29,2023-11-20 20:59:22,2023-11-20
879,406.0,2023-11-20 23:58:42,2023-11-20 23:58:55,invoices,2023112020583500453,card_acquiring_rub,21.71,460.79,482.5,Successful,2023-11-20,CreditCard,CreditCard - QiWi - RUB(redirect),482.5,2023-11-20 20:58:35,2023-11-20 20:59:06,2023-11-20
880,406.0,2023-11-20 23:58:24,2023-11-20 23:58:37,invoices,2023112020581900288,card_acquiring_rub,1.74,36.86,38.6,Successful,2023-11-20,CreditCard,CreditCard - QiWi - RUB(redirect),38.6,2023-11-20 20:58:19,2023-11-20 20:58:47,2023-11-20
881,406.0,2023-11-20 23:57:36,2023-11-20 23:58:29,invoices,2023112020572109744,card_acquiring_rub,34.74,737.26,772.0,Successful,2023-11-20,CreditCard,CreditCard - QiWi - RUB(redirect),772.0,2023-11-20 20:57:21,2023-11-20 20:58:40,2023-11-20


In [132]:
# Calculating the Net Receivables for Pagsmile and checking commission
merged_external_final["Net_Receivables"] = merged_external_final["Payer price"] -  merged_external_final["Shop fee"]
merged_external_final["Commission_4.5%"] = merged_external_final["Shop fee"] /  merged_external_final["Payer price"]
merged_external_final["Recheck commission"] = merged_external_final["Commission_4.5%"] <= 0.00451 

C:\Users\XCG\AppData\Local\Temp\ipykernel_10144\2316016786.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_external_final["Net_Receivables"] = merged_external_final["Payer price"] -  merged_external_final["Shop fee"]
C:\Users\XCG\AppData\Local\Temp\ipykernel_10144\2316016786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_external_final["Commission_4.5%"] = merged_external_final["Shop fee"] /  merged_external_final["Payer price"]
C:\Users\XCG\AppData\Local\Temp\ipykernel_10144\23160167

In [133]:
# Check if they charged more than 4.5%
merged_external_final["Recheck commission"].value_counts()
# As we see all commissions are less than 4.5%

False    55494
Name: Recheck commission, dtype: int64

In [134]:
merged_external_final["Payer price"].sum()

29599623.57

In [135]:
merged_external_final["Shop fee"].sum()

1331759.93

In [136]:
merged_external_final["Net_Receivables"].sum()

28263365.639999997

In [137]:
# Let's move this file to excel and csv
merged_external_final.to_excel("C:\\Users\\XCG\\Desktop\\For Automation\\Work_Automation\\external_recon_20231114_20231120_automated.xlsx", index = False)
#merged_external_final.to_csv("C:\\Users\\XCG\\Desktop\\For Automation\\Work_Automation\\merged_external_f_2_csv.csv", index = False)